In [ ]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
from timeit import timeit
from threshold import *

SAVE_TO = 'temp/'

In [ ]:
p = 0.2
r = 0.2

slope_t = timeit(lambda: threshold_slope(p,r,lengths=[4,5,6,8]), number=1)
thresh_t = timeit(lambda: above_threshold(p,r,large_L=10), number=1)

print(slope_t, thresh_t)

In [ ]:
slope_1000 = timeit(lambda: threshold_slope(p,r, samples=1000), number=1)
slope_2000 = timeit(lambda: threshold_slope(p,r, samples=2000), number=1)
thresh_1000 = timeit(lambda: above_threshold(p,r, samples=1000), number=1)
thresh_2000 = timeit(lambda: above_threshold(p,r, samples=2000), number=1)
print(slope_1000, slope_2000)
print(thresh_1000, thresh_2000)

In [ ]:
p_range = (0, 4)
r_range = (0, 4)
def inside_circle(p, r):
    return ((p-2)**2 + (r-2)**2) < 1

def cardioid(p,r):
    return (np.sqrt(p**2 + r**2) + np.cos(np.arctan(r/p))) < 1

engine = BoundarySearchEngine(p_range, r_range, split=2, max_depth=10, cache_dir=SAVE_TO, workers=8)
results = engine.boundary_search(inside_circle)
mp_time = timeit(lambda: engine.boundary_search(inside_circle, verbose=False), number=1)
seq_time = timeit(lambda: boundary_search(p_range, r_range, inside_circle, split=2, max_depth=10, cache_dir=SAVE_TO), number=1)
print(mp_time, seq_time)

In [ ]:
p_coords = []
r_coords = []
threshs = []
for (p, r), t in results.items():
    p_coords.append(p)
    r_coords.append(r)
    threshs.append(t)

fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.set_aspect('equal')
ax.set_xlabel('$p$ values')
ax.set_ylabel('$r$ values')

scat = ax.scatter(p_coords, r_coords, c=threshs, s=2, cmap='viridis')

In [ ]:
from matplotlib.animation import FuncAnimation, PillowWriter
from tqdm.auto import tqdm

p_range = (0, 1.5)
r_range = (0, 1.5)
inside_circle = lambda p, r: (p**2 + r**2) < 1
    
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8))
TICKS = 6

for ax in (ax1, ax2):
    ax.set_aspect('equal')
    ax.set_xlabel('$p$ values')
    ax.set_ylabel('$r$ values')

def update(frame):
    fig.suptitle(f'Recursion depth: {frame}')
    img, results = boundary_search(p_range, r_range, inside_circle, max_depth=frame)
    
    p_coords = []
    r_coords = []
    threshs = []
    for (p, r), t in results.items():
        p_coords.append(p)
        r_coords.append(r)
        threshs.append(t)

    artists = [
        ax1.scatter(p_coords, r_coords, c=threshs, s=2, cmap='viridis'),
        ax2.imshow(img)
    ]
    ax2.set_xticks(np.linspace(*ax2.get_xlim(), TICKS)),
    ax2.set_xticklabels(np.around(np.linspace(*p_range, TICKS), decimals=1)),
    ax2.set_yticks(np.linspace(*ax2.get_ylim(), TICKS)),
    ax2.set_yticklabels(np.around(np.linspace(*r_range, TICKS), decimals=1))
    return artists

anim = FuncAnimation(fig, update, frames=tqdm(range(1, 11)), interval=200, blit=True)
anim.save(os.path.join(SAVE_TO, 'boundary_search.gif'), writer=PillowWriter(fps=1), dpi=600)